# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:Alejandro Andres Velasco Santinato
*   Alumno 2:Daniela Leal Rivero
*   Alumno 3:John Briceño Diaz






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
import os
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

NOTA: esto es un fix para colab hay que ejecutarlo y cuando pida reiniciar el entorno se reinicia, despues de reiniciarlo simplemente ejecuten lo demas y deje esto como esta

In [ ]:

if IN_COLAB:
  !pip uninstall tensorflow -y
  !pip install tensorflow==2.15

In [3]:
%%capture
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5

else:
  %pip install numpy==1.23.5
  %pip install matplotlib==3.4.3
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.10.1
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [11]:
from PIL import Image
import numpy as np
import gym
#import matplotlib.pyplot as plt
#import pandas as pd
import keras
print(keras.__version__)
import tensorflow as tf

tf.keras.__version__ = keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy, MaxBoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, Callback

2.10.0


In [7]:
#Comprobando GPU
print(tf.config.experimental.list_physical_devices("GPU"))
print("Versión de TensorFlow:", tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Versión de TensorFlow: 2.10.1


#### Configuración base

In [8]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 8

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [9]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [12]:
class RewardLogger(Callback):
    """
    Callback para registrar las recompensas al final de cada episodio durante el test.

    :param episode_rewards: Lista para almacenar las recompensas de cada episodio.
    """
    def __init__(self):
        self.episode_rewards = []
    
    def on_episode_end(self, episode, logs={}):
        """
        Método llamado al final de cada episodio.

        :param episode: Número del episodio.
        :param logs: Diccionario de registros que contiene la recompensa del episodio.
        """
        self.episode_rewards.append(logs.get('episode_reward', 0))

In [ ]:
def graph_training(env_name, version_name, variables=['mean_q'], x_variable='nb_steps'):
    """
    Genera y guarda gráficos de la evolución de varias variables durante el entrenamiento.

    :param env_name: Nombre del entorno.
    :param version_name: Nombre de la versión del entrenamiento.
    :param variables: Lista de variables a graficar (por defecto ['mean_q']).
    :param x_variable: Variable del eje x para todas las gráficas (por defecto 'nb_steps').
    """
    
    with open(os.path.join(version_name, 'dqn_{}_{}_log.json'.format(env_name, version_name))) as f:
        data = json.load(f)

    num_vars = len(variables)
    cols = 2
    rows = (num_vars + 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Convertir a lista para unificar el manejo de los subplots
    colors = plt.cm.viridis(np.linspace(0, 1, num_vars))  # Colormap para diferentes colores

    df = pd.DataFrame(data)

    for ax, variable, color in zip(axes, variables, colors):
        if x_variable in df and variable in df:
            df.plot(x=x_variable, y=variable, ax=ax, legend=False, color=color)
            ax.set_title(f"Evolución - {variable}")
            ax.set_xlabel(x_variable.capitalize())
            ax.set_ylabel(variable.capitalize())
        else:
            ax.set_title(f"Variable '{variable}' or '{x_variable}' not found in data")
            ax.set_xlabel(x_variable.capitalize())
            ax.set_ylabel(variable.capitalize())

    # Eliminar subplots vacíos si el número de variables es impar
    if num_vars % cols != 0:
        fig.delaxes(axes[-1])

    plt.tight_layout()
    plt.savefig(os.path.join(version_name, 'dqn_{}_{}_{}.png'.format(env_name, version_name, "_".join(variables))))
    plt.show()

In [ ]:
def graph_testing(env_name, version_name, reward_logger):
    """
    Genera un gráfico de barras para las recompensas de cada episodio y una línea horizontal para el promedio.

    :param env_name: Nombre del entorno.
    :param version_name: Nombre de la versión del entrenamiento.
    :param reward_logger: Instancia de RewardLogger con las recompensas de los episodios.
    """
    # Verificar que haya recompensas registradas
    if not reward_logger.episode_rewards:
        print("No hay recompensas registradas en reward_logger.")
        return
    
    # Calcular el promedio de recompensas
    average_reward = sum(reward_logger.episode_rewards) / len(reward_logger.episode_rewards)
    
    # Crear el gráfico de barras
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(reward_logger.episode_rewards)), reward_logger.episode_rewards, alpha=0.7, label='Recompensas por episodio')
    
    # Añadir la línea del promedio
    plt.axhline(y=average_reward, color='r', linestyle='--', label=f'Promedio ({average_reward:.2f})')
    
    # Añadir etiquetas y título
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa')
    plt.title('Recompensas por episodio y promedio')
    plt.legend()

    # Guardar gráfico
    plt.savefig(os.path.join(version_name, 'dqn_{}_{}_test.png'.format(env_name, version_name)))

    # Mostrar el gráfico
    plt.show()

1. Implementación de la red neuronal

In [13]:
def build_model(input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=input_shape))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

input_shape = (WINDOW_LENGTH, INPUT_SHAPE[0], INPUT_SHAPE[1])
model = build_model(input_shape, nb_actions)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 8)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        16416     
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1606144   
                                                                 
 dense_1 (Dense)             (None, 6)                 3

## 2. Implementación de la solución DQN

Hiperparámetros

In [15]:

# Estrategia
POLICY_CLASSES = {
    'EpsGreedyQPolicy': EpsGreedyQPolicy(),
    'BoltzmannQPolicy': BoltzmannQPolicy(),
    'MaxBoltzmannQPolicy': MaxBoltzmannQPolicy()
}
hp = {
    # Procesador
    'MAX_FRAMES': 3, # Cantidad de frames consecutivos para realizar el máximo. 'None' para ignorar.
    'CROP_AREA': (33, 31, 124, 195), # Medidas para recortar el frame(left, upper, right, lower). 'None' para ignorar.
    'BW_THRESHOLD': 0, # Umbral para transformar el frame a blanco y negro. 'None' para ignorar.
    # Memoria
    'MEMORY_SIZE': 1000000, # Número máximo de experiencias que la memoria puede almacenar
    # Estrategia
    'POLICY': 'MaxBoltzmannQPolicy', # Opciones: EpsGreedyQPolicy, MaxBoltzmannQPolicy
    'EPS_MIN': 0.1, # Valor final de 'eps' (mínima exploración)
    'EPS_MAX': 1.0, # Valor inicial de 'eps' (máxima exploración)
    'EPS_TEST': 0.05, # Valor de 'eps' que se usará durante las pruebas
    'EPS_STEPS': 1000000, # Pasos entre EPS_MAX y EPS_MIN
    # Agente
    'STEPS_WARMUP': 50000, # Número de pasos antes de que el agente comience a entrenar
    'GAMMA': 0.99, # Factor de descuento de las recompensas futuras
    'TARGET_MODEL_UPDATE': 10000, # La frecuencia (en pasos) de actualización del modelo objetivo
    'TRAIN_INTERVAL': 8, # Cuántos pasos de interacción con el entorno se deben realizar antes de que el agente actualice sus pesos de red
    'DELTA_CLIP': 1, # Valor límite que se utiliza para recortar (limitar) las diferencias temporales que se usan para actualizar los pesos de la red
    'BATCH_SIZE': 64, # Cantidad de muestras que tomará de la memaria para realizar el entrenamiento
    # Entrenamiento
    'VISUALIZE_TRAINING': False, # Visualizar juego durante el entrenamiento
    'LEARNING_RATE': 0.00025, # Tasa de aprendizaje
    'CHECKPOINT_INTERVAL': 250000, #250000, # Intervalo de pasos después del cual se registrará el checkpoint
    'FILELOGGER_INTERVAL': 5000, # Intervalo de pasos después del cual se escribirá la información en el archivo
    'TRAINING_STEPS': 3000000, # Número total de pasos de entrenamiento
    'LOG_INTERVAL': 10000 # Intervalo de pasos en el que se registrará el progreso del entrenamiento
}

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
memory = SequentialMemory(limit=hp['MEMORY_SIZE'], window_length=WINDOW_LENGTH)

# Creando la policy
policy = LinearAnnealedPolicy(POLICY_CLASSES[hp['POLICY']], attr='eps', value_max=hp['EPS_MAX'], 
                              value_min=hp['EPS_MIN'], value_test=hp['EPS_TEST'], nb_steps=hp['EPS_STEPS'])

# Creando el agente de DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=AtariProcessor(), nb_steps_warmup=hp['STEPS_WARMUP'], gamma=hp['GAMMA'], 
               target_model_update=hp['TARGET_MODEL_UPDATE'], train_interval=hp['TRAIN_INTERVAL'], 
               delta_clip=hp['DELTA_CLIP'], batch_size=hp['BATCH_SIZE'])

# Compilando el agente
dqn.compile(Adam(learning_rate=hp['LEARNING_RATE']), metrics=['mae'])

In [20]:
# --- Parte de Entrenamiento con Checkpoints  ---

# Define el directorio para los checkpoints dentro de la carpeta drive_root
checkpoint_dir = os.path.join(drive_root, 'checkpoints')
# Crea el directorio si no existe (exist_ok=True evita errores si ya existe)
os.makedirs(checkpoint_dir, exist_ok=True)

# Define el nombre de archivo para guardar los pesos, incluyendo la ruta a tu carpeta de Drive
# {step} será reemplazado automáticamente por el número de pasos realizados durante el entrenamiento
checkpoint_weights_filename = os.path.join(checkpoint_dir, 'dqn_' + env_name + '_weights_{step}.h5f')

# Define los callbacks para guardar los pesos y registrar el progreso
# ModelIntervalCheckpoint guarda los pesos en intervalos especificados
# FileLogger registra el progreso del entrenamiento en un archivo JSON
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=hp['CHECKPOINT_INTERVAL'])]
callbacks += [FileLogger(os.path.join(drive_root, 'dqn_{}_log.json'.format(env_name)), interval=hp['FILELOGGER_INTERVAL'])]


# --- Iniciar Entrenamiento ---
print("\nIniciando el entrenamiento de DQN...")
# Puedes ajustar 'nb_steps' según tus recursos computacionales y la duración deseada del entrenamiento.
# Para un buen rendimiento en Atari, esto a menudo requiere millones de pasos.
# Comienza con un número menor como 500,000 para ver el progreso, luego auméntalo.
final_weights_filename = os.path.join(drive_root, 'dqn_{}_final_weights.h5f'.format(env_name))
if os.path.exists(final_weights_filename + '.data-00000-of-00001'):
    dqn.load_weights(final_weights_filename) # Carga los pesos finales guardados anteriormente
    print('Pesos cargados correctamente.')
else:
    print('No se encontraron pesos para cargar.')
    
dqn.fit(env, callbacks=callbacks, nb_steps=hp['TRAINING_STEPS'], log_interval=hp['LOG_INTERVAL'], 
        visualize=hp['VISUALIZE_TRAINING'], verbose=2)

# Después de entrenar, guarda los pesos finales
final_weights_filename = os.path.join(drive_root, 'dqn_{}_final_weights.h5f'.format(env_name))
#cargar pesos finales
dqn.save_weights(final_weights_filename, overwrite=True)
print(f"\nEntrenamiento finalizado. Pesos finales guardados en: {final_weights_filename}")


Iniciando el entrenamiento de DQN...
Pesos cargados correctamente.
Training for 2000000 steps ...
     906/2000000: episode: 1, duration: 6.781s, episode steps: 906, steps per second: 134, episode reward: 11.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.579 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1685/2000000: episode: 2, duration: 5.635s, episode steps: 779, steps per second: 138, episode reward: 11.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.619 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2149/2000000: episode: 3, duration: 3.220s, episode steps: 464, steps per second: 144, episode reward:  6.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.522 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2889/2000000: episode: 4, duration: 5.384s, episode steps: 740, steps per second: 137, episode reward: 10.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.557 [0.000, 5.000],  loss: --, mae

Testeo de la implementación

In [21]:
# --- Parte de prueba para calcular la recompensa media (después del entrenamiento) ---
print("\nIniciando la fase de prueba...")
# Carga los pesos finales para la prueba (o un checkpoint específico si quieres probar un modelo intermedio)
dqn.load_weights(final_weights_filename) # Carga los pesos finales guardados anteriormente
reward_logger = RewardLogger()
dqn.test(env, nb_episodes=10, visualize=False, callbacks=[reward_logger])

# Calcular el promedio de recompensas
average_reward = sum(reward_logger.episode_rewards) / len(reward_logger.episode_rewards)
print(f"Recompensa promedio en 100 episodios: {average_reward}")


Iniciando la fase de prueba...
Testing for 10 episodes ...
Episode 1: reward: 22.000, steps: 1013
Episode 2: reward: 25.000, steps: 928
Episode 3: reward: 22.000, steps: 1051
Episode 4: reward: 15.000, steps: 533
Episode 5: reward: 29.000, steps: 1743
Episode 6: reward: 19.000, steps: 915
Episode 7: reward: 31.000, steps: 1118
Episode 8: reward: 21.000, steps: 812
Episode 9: reward: 29.000, steps: 1045
Episode 10: reward: 17.000, steps: 791
Recompensa promedio en 100 episodios: 23.0


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---